# Merging of ``DataFrames``

In [1]:
from pandas import DataFrame
from viedoors import CADLoader, NPALoader, FileMerger, HMLoader
from viedoors import BSTLoader, FLTLoader, FMLoader

## Theoretical

In [ ]:
df_a = DataFrame({
    "x": [1,2,3,4,5],
    "merge": ["430 00A0213.01", "430 00A0213.01", "430 00A0213.03", "430 00A0213.02", None]
})

df_b = DataFrame({
    "y": ["a", "b", "c", "d"],
    "merge": ["430 00A0213.00", "430 00A0213.01", "430 00A0213.02", "430 00A0213.03"]
})

df_c = DataFrame({
    "z": ["x", "y"],
    "merge": ["430 00A0213.01", "430 00A0213.03"]
})

In [3]:
fm = FileMerger(files=[df_a, df_b, df_c],)

In [4]:
fm.get_data_merge()

,x,merge,y,z
0,1,430 00A0213.01,b,x
1,2,430 00A0213.01,b,x
2,3,430 00A0213.03,d,y
3,4,430 00A0213.02,c,NaN
4,5,None,NaN,NaN


In [5]:
fm.merge_success_rate()

1.0

## Real Data

In [3]:
cad = CADLoader(file="data/420_gesamt_20250122.xlsx", title="CAD")
npa = NPALoader(file="data/NPA_Tuer Aufnahme Obj420.xlsx", title="NPA")
hm = HMLoader(file="data/Schrack_HM_Obj420.xls", title="HM")
bst = BSTLoader(file="data/Sisando_BST_Obj420.xlsx", title="BST")
flt = FLTLoader(file="data/Sisando_FLT_Obj420.xlsx", title="FLT")
fm = FMLoader(file="data/BSTüren_Filemaker_11.04.22.xlsx", title="FM")

df_npa = npa.get_data(prefixed=True)
df_cad = cad.get_data(prefixed=True)
df_hm = hm.get_data(prefixed=True)
df_bst = bst.get_data(prefixed=True)
df_flt = flt.get_data(prefixed=True)
df_fm = fm.get_data(prefixed=True)

merger = FileMerger(files=[df_cad, df_npa, df_hm, df_bst, df_flt, df_fm], how="left")
merge = merger.get_data_merge()

In [5]:
merge.to_excel("test.xlsx")

In [6]:
merger.merge_success_rate()

0.06663560111835974